In [1]:
import pandas as pd
import numpy as np
import os
os.environ['MKL_NUM_THREADS'] = '1'
from collections import Counter
import itertools
import json
import scipy.sparse as sparse
import pickle
import torch
import misc.util as util
from torch import nn

import importlib
from torch.utils.data import DataLoader
from misc.loader import AEDataset
from misc.util import *
from tqdm.auto import tqdm
from eval.rec_eval import *
import neuralsort.pl as pl
from models.loss import neuPrecLoss
from misc.loader import RecDataset
import models
from models.loss import *
import models.mf
import models.mf
import neuralsort.neuralobjs

In [2]:
with open(os.path.join("data", "parsed", "ml-20m"), 'rb') as f:
    a,b,c, tr_tr, tr_val, te = pickle.load(f)

In [7]:
!ls -al saved_models/ml-20m

total 985820
drwxrwxr-x 2 ita ita      4096  7월 21 23:36 .
drwxrwxr-x 5 ita ita      4096  7월 20 00:28 ..
-rw-rw-r-- 1 ita ita 336018978  7월 24 12:10 dae_ndcg5
-rw-rw-r-- 1 ita ita 336018978  7월 24 12:10 dae_prec5
-rw-rw-r-- 1 ita ita 168045090  7월 24 13:39 dae_rc50
-rw-rw-r-- 1 ita ita  29666020  7월 22 07:10 vae_ndcg5
-rw-rw-r-- 1 ita ita  29666020  7월 22 07:10 vae_prec5
-rw-rw-r-- 1 ita ita  29666013  7월 22 06:27 vae_rc50
-rw-rw-r-- 1 ita ita  26789773  7월 22 00:26 wmf_ndcg5
-rw-rw-r-- 1 ita ita  26793691  7월 22 00:25 wmf_prec5
-rw-rw-r-- 1 ita ita  26784097  7월 22 00:26 wmf_rc50


In [8]:
best_model = torch.load("saved_models/ml-20m/wmf_prec5")

In [9]:
ae_dataset = AEDataset(tr_tr)

In [15]:
model = models.mf.mfrec(*tr_tr.shape, emb_size= best_model['item_factors'].shape[-1]).cuda()
model.u_emb.weight.data.copy_(torch.from_numpy(best_model['user_factors']))
model.i_emb.weight.data.copy_(torch.from_numpy(best_model['item_factors']))
print(ranking_metrics_at_k(model, tr_tr, te, K=5, num_threads=4))
print(ranking_metrics_at_k(model, tr_tr, te, K=10, num_threads=4))

{'precision': 0.06289999999999984, 'recall': 0.06125415651886377, 'map': 0.11893097222222339, 'ndcg': 0.07423875357580718}
{'precision': 0.054770000000001665, 'recall': 0.09992439818876335, 'map': 0.12580026114890383, 'ndcg': 0.08783206817564923}


In [13]:
importlib.reload(models.mf)
loader = DataLoader(ae_dataset, batch_size=128, shuffle=True, num_workers=6)
model = models.mf.mfrec(*tr_tr.shape, emb_size= best_model['item_factors'].shape[-1]).cuda()
model.u_emb.weight.data.copy_(torch.from_numpy(best_model['user_factors']))
model.i_emb.weight.data.copy_(torch.from_numpy(best_model['item_factors']))
optimizer = torch.optim.RMSprop(model.parameters(), lr=1 * 1e-5, weight_decay=0)
sc = neuralsort.neuralobjs.SC()
import models.lrloss as lrloss

In [14]:
last_prec = -1
for epoch in (range(1, 10 + 1)):
    tot_loss = 0
    for uid, rowl in tqdm(loader):
        row = rowl.float().cuda()
        uid = uid.cuda() 
        scores = model.fow2(uid)
        #y_hat, indices = torch.topk(scores, 300)
        #y_true = row.gather(1, indices)
        #loss  = lrloss.approxNDCGLoss(y_hat, y_true)
        loss  = neuPrecLoss(sc, scores, row, topk=300, k=3, tau=20.0, use_top=True).mean()
        loss.backward()
        optimizer.step()

        
    if (epoch % 1 == 0):
        metric = ranking_metrics_at_k(model, tr_tr, tr_val, K=5, num_threads=4)
        print(metric)
        pp = metric['precision']
        print(ranking_metrics_at_k(model, tr_tr, te, K=5, num_threads=4))
        print(ranking_metrics_at_k(model, tr_tr, te, K=10, num_threads=4))
        if pp < last_prec:
            break
        last_prec = pp 
        
        


{'precision': 0.06373999999999991, 'recall': 0.0640980881736164, 'map': 0.12623319444444547, 'ndcg': 0.07744373214625845}
{'precision': 0.06552000000000018, 'recall': 0.06299061051305847, 'map': 0.12382902777777884, 'ndcg': 0.07709887504301007}
{'precision': 0.05629000000000173, 'recall': 0.10190941270776055, 'map': 0.13124878533635684, 'ndcg': 0.0902798910836943}



{'precision': 0.06570000000000012, 'recall': 0.06496578413660888, 'map': 0.13009263888888992, 'ndcg': 0.07929501875332638}
{'precision': 0.06634000000000029, 'recall': 0.06313218365422817, 'map': 0.12678722222222322, 'ndcg': 0.07800259116008855}
{'precision': 0.05695000000000193, 'recall': 0.10287182086188827, 'map': 0.13368090929705173, 'ndcg': 0.09112916718634595}



{'precision': 0.06626000000000011, 'recall': 0.06472269353503209, 'map': 0.13272638888889018, 'ndcg': 0.07981093004893332}
{'precision': 0.06640000000000024, 'recall': 0.06291374925444006, 'map': 0.1295416666666675, 'ndcg': 0.07850866341162986}
{'precision': 0.05669000000000203, 'recall': 0.10172465338152772, 'map': 0.13654796712018097, 'ndcg': 0.09104409700706467}



{'precision': 0.06614000000000016, 'recall': 0.0638515244131565, 'map': 0.13355375000000108, 'ndcg': 0.07961361472140963}
{'precision': 0.0658600000000002, 'recall': 0.06202572118309059, 'map': 0.13032000000000077, 'ndcg': 0.07809392790919469}
{'precision': 0.05631000000000206, 'recall': 0.10021200470651885, 'map': 0.13807607823129217, 'ndcg': 0.09027882214159944}


In [1]:
import models.mf
import neuralsort.neuralobjs
loader = DataLoader(ae_dataset, batch_size=128, shuffle=True, num_workers=6)
model = models.mf.mfrec(*tr_tr.shape, emb_size=pt_wmf['user_factors'].shape[1]).cuda()
model.u_emb.weight.data.copy_(torch.from_numpy(pt_wmf['user_factors']))
model.i_emb.weight.data.copy_(torch.from_numpy(pt_wmf['wmf_item_factors']))
print(ranking_metrics_at_k(model, tr_tr, tr_val, K=30, num_threads=4))
optimizer = torch.optim.Adam(model.parameters(), lr=1 * 1e-6, weight_decay=1e-2)
sc = neuralsort.neuralobjs.SC()

NameError: name 'DataLoader' is not defined

In [ ]:
for epoch in (range(1, 30 + 1)):
    tot_loss = 0
    for uid, rowl in tqdm(loader):
        row = rowl.float().cuda()
        uid = uid.cuda() 
        scores = model.forward(uid)
        loss  = neuPrecLoss(sc, scores, row, topk=500, k=10, tau=15.0, use_top=True).mean()
        loss.backward()
        optimizer.step()
    if (epoch % 1 == 0):
        print(ranking_metrics_at_k(model, tr_tr, tr_val, K=30, num_threads=4))

In [ ]:
row.shape

In [ ]:
rd = torch.randint(row.shape[0], row.shape[1], size=(256, 1234))

In [ ]:
_row = row.gather(1, rd.cuda())
_score = scores.gather(1, rd.cuda())

In [ ]:
_row.shape

In [ ]:
_score.shape